# Creating Geoscience Objects

This notebook demonstrates how to create various geoscience objects using the typed object interfaces
in the `evo-objects` package.

Covered objects:
- PointSet
- RegularMasked3DGrid
- Variogram (using low-level API)

## Authentication

In [ ]:
from evo.notebooks import ServiceManagerWidget

manager = await ServiceManagerWidget.with_auth_code(
    client_id="your-client-id",  # Replace with your client ID
    base_uri="https://qa-ims.bentley.com",
    discovery_url="https://int-discover.test.api.seequent.com",
    cache_location="./notebook-data",
).login()

## Creating a PointSet

PointSets represent collections of 3D points with associated attributes.
Common uses include drill hole data, sample locations, and survey points.

In [ ]:
import uuid
import numpy as np
import pandas as pd
from evo.notebooks import display_object_links
from evo.objects.typed import EpsgCode, PointSet, PointSetData

# Generate sample point data
n_points = 100
np.random.seed(42)

# Create points in a 1000x1000x100 domain
x = np.random.uniform(0, 1000, n_points)
y = np.random.uniform(0, 1000, n_points)
z = np.random.uniform(0, 100, n_points)

# Create attributes
elevation = z + np.random.normal(0, 5, n_points)  # Elevation with noise
grade = np.random.exponential(2, n_points)  # Grade values
rock_type = np.random.choice(["sandstone", "limestone", "shale"], n_points)

# Create a DataFrame with coordinates and attributes
locations_df = pd.DataFrame({
    "x": x,
    "y": y,
    "z": z,
    "elevation": elevation,
    "grade": grade,
    "rock_type": rock_type,
})

print("Sample data:")
locations_df.head()

In [ ]:
# Create the PointSet object
pointset_data = PointSetData(
    name=f"Sample Points - {uuid.uuid4()}",
    coordinate_reference_system=EpsgCode(32632),  # UTM zone 32N
    locations=locations_df,
)

pointset = await PointSet.create(manager, pointset_data)

print(f"Created PointSet: {pointset.name}")
print(f"Number of points: {len(locations_df)}")
print(f"Attributes: {list(locations_df.columns[3:])}")
display_object_links(pointset, label="Created PointSet")

## Creating a RegularMasked3DGrid

Regular 3D grids are used for block models and volumetric data.
A mask allows defining which cells are active/valid.

In [ ]:
from evo.objects.typed import Point3, RegularMasked3DGrid, RegularMasked3DGridData, Size3d, Size3i
from evo.objects.typed import Rotation as GridRotation

# Define grid parameters
nx, ny, nz = 20, 20, 10  # Number of cells
cell_size = 50.0  # Size of each cell in meters

# Calculate total cells
total_cells = nx * ny * nz
print(f"Grid dimensions: {nx} x {ny} x {nz} = {total_cells} cells")

# Create a mask (True = active cell)
# Cells are ordered x-fastest, then y, then z
mask = np.ones(total_cells, dtype=bool)

# Mask out a corner region to create an irregular shape
for zi in range(nz // 2):
    for yi in range(ny // 2):
        for xi in range(nx // 2):
            idx = xi + yi * nx + zi * nx * ny
            mask[idx] = False

print(f"Active cells: {int(mask.sum())} ({100 * mask.sum() / total_cells:.1f}%)")

In [ ]:
# Create the grid object
grid_data = RegularMasked3DGridData(
    name=f"Sample Grid - {uuid.uuid4()}",
    coordinate_reference_system=EpsgCode(32632),
    origin=Point3(0, 0, 0),
    size=Size3i(nx, ny, nz),
    cell_size=Size3d(cell_size, cell_size, cell_size),
    rotation=GridRotation(0, 0, 0),
    mask=mask,
    cell_data=None,  # No initial attributes
)

grid = await RegularMasked3DGrid.create(manager, grid_data)

print(f"Created Grid: {grid.name}")
print(f"Origin: {grid.origin}")
print(f"Cell size: {grid.cell_size}")
print(f"Bounding box: {grid.bounding_box}")
display_object_links(grid, label="Created Grid")

## Creating a Variogram

Variograms model spatial correlation and are essential for geostatistical analysis.

> **Note:** A typed Variogram class will be available in a future release.
> For now, we use the low-level `ObjectAPIClient`.

In [ ]:
from evo.objects import ObjectAPIClient

object_client = ObjectAPIClient.from_context(manager)

# Define a spherical variogram model
variogram_metadata = {
    "schema": "/objects/variogram/1.1.0/variogram.schema.json",
    "name": f"Sample Variogram - {uuid.uuid4()}",
    "nugget": 0.1,  # Random variation at zero distance
    "sill": 1.0,    # Total variance
    "is_rotation_fixed": True,
    "structures": [
        {
            "variogram_type": "spherical",  # Options: spherical, exponential, gaussian
            "contribution": 0.9,  # sill - nugget
            "anisotropy": {
                "ellipsoid_ranges": {
                    "major": 200.0,      # Range in major direction
                    "semi_major": 200.0, # Range in semi-major direction
                    "minor": 100.0,      # Range in minor direction
                },
                "rotation": {
                    "dip_azimuth": 0.0,
                    "dip": 0.0,
                    "pitch": 0.0,
                },
            },
        }
    ],
    "attribute": "grade",  # The attribute this variogram models
    "domain": "all",
}

variogram = await object_client.create_geoscience_object(
    path=f"/variograms/sample-variogram-{uuid.uuid4()}.json",
    object_dict=variogram_metadata,
)

print(f"Created Variogram: {variogram.name}")
print(f"Nugget: {variogram_metadata['nugget']}")
print(f"Sill: {variogram_metadata['sill']}")
print(f"Type: {variogram_metadata['structures'][0]['variogram_type']}")
display_object_links(variogram, label="Created Variogram")

## Verifying Created Objects

Load the objects back and verify they were created correctly.

In [ ]:
# Reload the pointset
loaded_pointset = await PointSet.from_reference(manager, pointset.metadata.url)

print(f"Loaded PointSet: {loaded_pointset.name}")
print(f"CRS: {loaded_pointset.coordinate_reference_system}")

# Get locations as DataFrame
df = await loaded_pointset.locations.as_dataframe()
print(f"\nData preview:")
df.head()

In [ ]:
# Reload the grid
loaded_grid = await RegularMasked3DGrid.from_reference(manager, grid.metadata.url)

print(f"Loaded Grid: {loaded_grid.name}")
print(f"Dimensions: {loaded_grid.size}")
print(f"Cell size: {loaded_grid.cell_size}")
print(f"Bounding box: {loaded_grid.bounding_box}")

## Summary

This notebook demonstrated:

1. **PointSet creation** - Using `PointSetData` with a pandas DataFrame
2. **RegularMasked3DGrid creation** - Defining grid geometry with a cell mask
3. **Variogram creation** - Using the low-level API (typed class coming soon)

These objects can now be used as inputs for compute tasks like kriging.
